# 0. Import libraries

In [74]:
# Basic
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 1. Load data

## 1.1 Loading

In [81]:
def load_aqms_data(filepath):
    """
    Load and process AQMS Excel file containing asset price data.
    
    Args:
        filepath (str): Path to AQMS Excel file
        
    Returns:
        pd.DataFrame: Processed DataFrame with datetime index
    """
    xls = pd.ExcelFile(filepath)
    merged_df = None
    
    for sheet_name in xls.sheet_names:
        # Define sheet-specific parameters
        if sheet_name == "Equity":
            skiprows = [0, 1, 3, 4]
        else:
            skiprows = [0, 1, 2, 4, 5]
            
        # Read sheet with custom parameters
        df = pd.read_excel(
            xls,
            sheet_name=sheet_name,
            skiprows=skiprows,
            header=0
        ).copy()
        
        # Process date column
        date_col = df.columns[0]
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        df = df.dropna(subset=[date_col]).sort_values(date_col)
        
        # Prefix columns with sheet name to avoid collisions
        df.columns = [f"{sheet_name}_{col}" if col != date_col else col 
                     for col in df.columns]
        
        # Rename date column and merge
        df = df.rename(columns={date_col: 'Date'})
        if merged_df is None:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on='Date', how='outer')
    
    # Final processing
    aqms_df = merged_df.sort_values('Date').reset_index(drop=True)
    aqms_df['Date'] = pd.to_datetime(aqms_df['Date'])
    aqms_df.set_index('Date', inplace=True)
    
    return aqms_df


def load_business_cycle_data(filepath):
    """
    Load and process Business Cycle Excel file containing GDP and CPI data.
    
    Args:
        filepath (str): Path to Business Cycle Excel file
        
    Returns:
        pd.DataFrame: Processed DataFrame with year-end datetime index
    """
    sheet_names = ['GDP', 'CPI']
    merged_df = None
    
    for sheet in sheet_names:
        # Read sheet and drop rows with missing country names
        df = pd.read_excel(filepath, sheet_name=sheet, skiprows=[1])
        df = df[df.iloc[:, 0].notna()].reset_index(drop=True)
        
        # Rename first column to 'Country'
        df = df.rename(columns={df.columns[0]: 'Country'})
        
        # Convert to long format
        long_df = df.melt(
            id_vars='Country', 
            var_name='Year', 
            value_name='Value'
        )
        long_df['Year'] = long_df['Year'].astype(str)
        
        # Create feature names and pivot to wide format
        long_df['Feature'] = f"{sheet}_" + long_df['Country']
        pivot_df = long_df.pivot(
            index='Year', 
            columns='Feature', 
            values='Value'
        )
        
        # Merge sheets
        if merged_df is None:
            merged_df = pivot_df
        else:
            merged_df = merged_df.join(pivot_df, how='outer')
    
    # Set year-end datetime index
    bc_df = merged_df.sort_index()
    bc_df.index = pd.to_datetime(bc_df.index.astype(str)) + pd.offsets.YearEnd(0)
    
    return bc_df


def preprocess_asset_data(aqms_df):
    """
    Preprocess asset data to calculate annual returns and yields.
    
    Args:
        aqms_df (pd.DataFrame): Raw AQMS DataFrame
        
    Returns:
        pd.DataFrame: Processed annual returns and yields
    """
    # Define asset categories
    equity = ['US', 'UK', 'Japan', 'Hong Kong', 'Canada', 'Euro', 'Switzerland', 'New Zealand', 'Australia']
    
    int_future = ['SFRA Comdty', 'SFR1YZ2 Comdty', 'SFR1YZ3 Comdty', 
                  'SFR1YZ4 Comdty', 'SFR1YZ5 Comdty', 'SFR1YZ6 Comdty', 
                  'SFR2YZ2 Comdty']
    
    raw = ['JP', 'EU', 'HK', 'CH', 'CA', 'AU', 'NZ']
    currency = raw + ['GBP']
    gov_bond = raw + ['US', 'UK']
    
    # Initialize lists for asset columns
    gb2y, gb10y, curr, eqt, irf = [], [], [], [], []
    
    # Categorize columns
    for name in aqms_df.columns:
        for e in equity:
            if (e in name) and ('Equity' in name) and ('Pan' not in name):
                eqt.append(name)
                
        for g in gov_bond:
            if (g in name) and ('2' in name) and ('Bond' in name):
                gb2y.append(name)
            if (g in name) and ('10' in name) and ('Bond' in name):
                gb10y.append(name)
                
        for c in currency:
            if (c in name) and ('Curncy' in name):
                curr.append(name)
                
        for i in int_future:
            if (i in name) and ('Future' in name):
                irf.append(name)
    
    # Process interest rate columns
    rate_mapping = {
        'US': ['FDTR Index'],
        'UK': ['UKBRBASE Index'],
        'JP': ['BOJDTR Index'],
        'EU': ['EURR002W Index'],
        'HK': ['PRIMHK Index'],
        'CA': ['CABROVER Index'],
        'AU': ['RBATCTR Index'],
        'NZ': ['NZOCR Index'],
        'CH': ['SZNBPOLR Index']
    }
    
    # Create renaming dictionary
    renaming_dict = {}
    for col in aqms_df.columns:
        for abbr, indices in rate_mapping.items():
            for index in indices:
                if index in col:
                    renaming_dict[col] = col.replace(index, abbr)
    
    # Rename columns and get interest rate columns
    aqms_renamed = aqms_df.rename(columns=renaming_dict)
    ir = aqms_renamed[list(renaming_dict.values())]
    
    # Combine all asset data
    assets = pd.concat([
        aqms_renamed[eqt], 
        aqms_renamed[gb2y], 
        aqms_renamed[gb10y], 
        aqms_renamed[curr], 
        aqms_renamed[irf]
    ], axis=1)
    
    # Forward and backward fill missing values
    assets_imputed = assets.ffill().bfill()
    ir_imputed = ir.ffill().bfill()
    
    # Calculate annual returns for price-like assets
    price_cols = [col for col in assets.columns 
                 if col.startswith('Equity_') 
                 or col.startswith('Currency_') 
                 or 'IR Future' in col]
    
    log_returns = np.log(assets_imputed[price_cols] / assets_imputed[price_cols].shift(1))
    annual_log_returns = log_returns.resample('Y').sum()
    annual_returns = np.exp(annual_log_returns) - 1
    
    # Calculate annual average yields for bonds and interests
    annual_yields = assets_imputed[gb2y + gb10y].resample('Y').mean() / 100
    ir_annual = ir_imputed.resample('Y').mean() / 100
    
    # Combine returns and yields
    annual_df = pd.concat([annual_returns, annual_yields], axis=1)
    
    return annual_df, ir_annual


def preprocess_macro_factors(bc_df, annual_df, ir):
    """
    Preprocess macro factors from business cycle data and asset returns.
    
    Args:
        bc_df (pd.DataFrame): Business cycle DataFrame
        annual_df (pd.DataFrame): Annual asset returns DataFrame
        ir: df of imputed and annual average dataframe
        
    Returns:
        pd.DataFrame: Processed macro factors DataFrame
    """
    # Initialize macro factors DataFrame
    mf = pd.DataFrame(index=annual_df.index)
    
    # Country code mapping for consistent naming
    country_to_abbr = {
        'US': 'US',
        'United States': 'US',
        'UK': 'UK',
        'United Kingdom': 'UK',
        'Japan': 'JP',
        'Hong Kong': 'HK',
        'Hong Kong SAR': 'HK',
        'Canada': 'CA',
        'Euro': 'EU',
        'European Union': 'EU',
        'Switzerland': 'CH',
        'Australia': 'AU',
        'New Zealand': 'NZ',
        'Switzerland': 'CH'
    }
    
    # Rename columns in business cycle data
    renamed_columns = {}
    for col in bc_df.columns:
        if col.startswith('GDP_'):
            for country, abbr in country_to_abbr.items():
                if col == f'GDP_{country}':
                    renamed_columns[col] = f'GDP_{abbr}'
        elif col.startswith('CPI_'):
            for country, abbr in country_to_abbr.items():
                if col == f'CPI_{country}':
                    renamed_columns[col] = f'CPI_{abbr}'
    
    bc_renamed = bc_df.rename(columns=renamed_columns)
    bc_renamed = bc_renamed[renamed_columns.values()]

    # Convert business cycle data to numeric, handling 'no data' entries
    for col in bc_renamed.columns:
        if col.startswith(('GDP_', 'CPI_')):
            # Convert to numeric, coercing errors to NaN
            bc_renamed[col] = pd.to_numeric(bc_renamed[col], errors='coerce')
            # Divide by 100 only for valid numeric values
            bc_renamed[col] = bc_renamed[col] / 100
    
    # Calculate excess returns (equity returns minus risk-free rate)
    countries = ['US', 'UK', 'Japan', 'Hong Kong', 'Canada', 'Euro', 'Australia', 'New Zealand','Switzerland']
    
    # Calculate excess returns for each country
    for country in countries:
        equity_return = annual_df[f'Equity_{country}']
        risk_free_rate = ir[f'Interest Rates_{country_to_abbr[country]}']
        mf[f'Excess_Return_{country_to_abbr[country]}'] = equity_return - risk_free_rate
    
    # Add business cycle data (GDP and CPI)
    mf = mf.join(bc_renamed, how='left')
    
    # Add currency returns
    currency_cols = [col for col in annual_df.columns if col.startswith("Currency_")]
    mf = mf.join(annual_df[currency_cols], how='left')
    
    # Add monetary policy proxies (2-year bond yields)
    monetary_policy_cols = [col for col in annual_df.columns 
                          if col.startswith("Bond Yield 2Y_")]
    mf = mf.join(annual_df[monetary_policy_cols], how='left')
    
    # Filter to desired time period (1980-2025)
    mf = mf[(mf.index.year >= 1980) & (mf.index.year <= 2025)]
    
    return mf

In [82]:
# data_dir = "C:\\Users\\DavidJIA\\Desktop\\IC RMFE\\Term 3\\Applied Macro Trading Strategy\\CourseWork\\datasets\\"
data_dir = './'
# Load and process data
aqms_data = load_aqms_data(data_dir + 'AQMS.xlsx')
bc_data = load_business_cycle_data(data_dir + 'Business Cycle.xls')

# Preprocess asset data
ar, ir = preprocess_asset_data(aqms_data)
print(ar.columns)

# Preprocess macro factors
mf = preprocess_macro_factors(bc_data, ar, ir)

Index(['Equity_US', 'Equity_UK', 'Equity_Japan', 'Equity_Hong Kong',
       'Equity_Canada', 'Equity_Euro', 'Equity_Switzerland',
       'Equity_Australia', 'Equity_New Zealand', 'Currency_GBP Curncy',
       'Currency_JPY Curncy', 'Currency_EUR Curncy', 'Currency_HKD Curncy',
       'Currency_CHF Curncy', 'Currency_CAD Curncy', 'Currency_AUD Curncy',
       'Currency_NZD Curncy', 'IR Future_SFRA Comdty',
       'IR Future_SFR1YZ2 Comdty', 'IR Future_SFR1YZ3 Comdty',
       'IR Future_SFR1YZ4 Comdty', 'IR Future_SFR1YZ5 Comdty',
       'IR Future_SFR1YZ6 Comdty', 'IR Future_SFR2YZ2 Comdty',
       'Bond Yield 2Y_USGG2YR Index', 'Bond Yield 2Y_GUKG2 Index',
       'Bond Yield 2Y_GTJPY2Y Govt', 'Bond Yield 2Y_GTHKD2Y Govt',
       'Bond Yield 2Y_GTCAD2Y Govt', 'Bond Yield 2Y_GTCHF2Y Govt',
       'Bond Yield 2Y_GTAUD2Y Govt', 'Bond Yield 2Y_GTNZD2Y Govt',
       'Bond Yield 2Y_GTEURTR2Y Govt', 'Bond Yield 10Y_USGG10YR Index',
       'Bond Yield 10Y_GUKG10 Index', 'Bond Yield 10Y_GTJPY10Y

In [83]:
# Unify column names
def standardize_column_names(df):
    """
    Standardize column names to use consistent two-digit country abbreviations.
    
    Args:
        df (pd.DataFrame): DataFrame with columns to be renamed
        
    Returns:
        pd.DataFrame: DataFrame with standardized column names
    """
    # Country name to two-letter abbreviation mapping
    country_map = {
        'US': 'US',
        'UK': 'UK',
        'Japan': 'JP',
        'Hong Kong': 'HK',
        'Canada': 'CA',
        'Pan-Europe': 'EU',
        'Euro': 'EU',
        'Switzerland': 'CH',
        'Australia': 'AU',
        'New Zealand': 'NZ',
        'GBP': 'UK',  # Currency special cases
        'JPY': 'JP',
        'EUR': 'EU',
        'HKD': 'HK',
        'CHF': 'CH',
        'CAD': 'CA',
        'AUD': 'AU',
        'NZD': 'NZ'
    }
    
    new_columns = []
    for col in df.columns:
        parts = col.split('_')
        
        # Handle different column patterns
        if col.startswith('Equity_'):
            country = parts[1]
            new_col = f"Equity_{country_map.get(country, country)}"
            
        elif col.startswith('Currency_'):
            currency = parts[1].split()[0]  # Get currency code before "Curncy"
            new_col = f"Currency_{country_map.get(currency, currency)}"
            
        elif col.startswith('IR Future_'):
            # Keep IR Future columns as-is (they're contracts, not countries)
            new_col = col
            
        elif col.startswith(('Bond Yield 2Y_', 'Bond Yield 10Y_')):
            # Extract country code from bond yield columns
            bond_part = parts[1]
            if bond_part.startswith('GT'):  # Handle GT-prefixed bonds (e.g., GTJPY)
                country_code = bond_part[2:4]  # Gets JP from GTJPY
            elif bond_part.startswith('GU'):  # Handle UK bonds (GUKG)
                country_code = 'UK'
            elif bond_part.startswith('US'):  # US bonds
                country_code = 'US'
            elif bond_part.startswith('HK'):  # Hong Kong bonds
                country_code = 'HK'
            else:
                # Fallback - take first 2 characters
                country_code = bond_part[:2]
            new_col = f"{parts[0].replace(' ','')}_{country_code}"
    
        elif col.startswith('Excess_Return_'):
            country = parts[2]  # Changed from parts[1] to parts[2]
            new_col = f"ExcessReturn_{country_map.get(country, country)}"
            
        elif col.startswith(('GDP_', 'CPI_')):
            country = parts[1]
            new_col = f"{parts[0]}_{country_map.get(country, country)}"
            
        else:
            new_col = col  # Leave unchanged if no pattern matches
            
        new_columns.append(new_col)
    
    # Apply the new column names
    renamed_df = df.copy()
    renamed_df.columns = new_columns
    
    return renamed_df


# Apply to both DataFrames
ar = standardize_column_names(ar)
mf = standardize_column_names(mf)

# Display sample results
print("Asset Returns columns sample:")
print(ar.columns)  # First 10 columns

print("\nMacro Factors columns sample:")
print(mf.columns)  # First 10 columns

Asset Returns columns sample:
Index(['Equity_US', 'Equity_UK', 'Equity_JP', 'Equity_HK', 'Equity_CA',
       'Equity_EU', 'Equity_CH', 'Equity_AU', 'Equity_NZ', 'Currency_UK',
       'Currency_JP', 'Currency_EU', 'Currency_HK', 'Currency_CH',
       'Currency_CA', 'Currency_AU', 'Currency_NZ', 'IR Future_SFRA Comdty',
       'IR Future_SFR1YZ2 Comdty', 'IR Future_SFR1YZ3 Comdty',
       'IR Future_SFR1YZ4 Comdty', 'IR Future_SFR1YZ5 Comdty',
       'IR Future_SFR1YZ6 Comdty', 'IR Future_SFR2YZ2 Comdty',
       'BondYield2Y_US', 'BondYield2Y_UK', 'BondYield2Y_JP', 'BondYield2Y_HK',
       'BondYield2Y_CA', 'BondYield2Y_CH', 'BondYield2Y_AU', 'BondYield2Y_NZ',
       'BondYield2Y_EU', 'BondYield10Y_US', 'BondYield10Y_UK',
       'BondYield10Y_JP', 'BondYield10Y_HK', 'BondYield10Y_CA',
       'BondYield10Y_CH', 'BondYield10Y_AU', 'BondYield10Y_NZ'],
      dtype='object')

Macro Factors columns sample:
Index(['ExcessReturn_US', 'ExcessReturn_UK', 'ExcessReturn_JP',
       'ExcessReturn_HK'

One small manual: here main output is ar(asset return) and mf(macro factors).  

ar: annualized geo mean return of each assets, imputed due to severe lack of interest rate future asset data  

mf: annual return of macro factors

all labels are encoded in column name, like "Equity_JP" or "GDP_AU"

In [84]:
def process_ir_futures(ar_df):
    """
    Process interest rate futures data by:
    1. Dropping existing IR Future columns
    2. Generating pseudo-data for each country
    
    Args:
        ar_df (pd.DataFrame): Asset returns DataFrame
        
    Returns:
        pd.DataFrame: Processed DataFrame with pseudo IR Futures
    """
    # 1. Drop all existing IR Future columns
    ir_future_cols = [col for col in ar_df.columns if 'IR Future' in col]
    ar_processed = ar_df.drop(columns=ir_future_cols)
    
    # 2. Generate pseudo-data for each country
    countries = ['US', 'UK', 'JP', 'HK', 'CA', 'EU', 'CH', 'AU', 'NZ']
    
    for country in countries:
        col_name = f'IRFutures_{country}'
        
        # Generate random returns between -0.02 and 0.02 (2%)
        pseudo_data = np.random.uniform(low=-0.02, high=0.02, size=len(ar_df))
        
        # Add slight autocorrelation to make it more realistic
        for i in range(1, len(pseudo_data)):
            pseudo_data[i] = 0.7*pseudo_data[i-1] + 0.3*pseudo_data[i]
        
        ar_processed[col_name] = pseudo_data
    
    return ar_processed

# Apply the processing
ar = process_ir_futures(ar)

# Verify the result
print("Columns after processing:")
print([col for col in ar.columns if 'IRFutures' in col])
print("\nSample pseudo data:")
# print(ar[[f'IRFutures_US', f'IRFutures_JP']].head())

Columns after processing:
[]

Sample pseudo data:


KeyError: "None of [Index(['IRFutures_US', 'IRFutures_JP'], dtype='object')] are in the [columns]"

In [86]:
ar.columns

Index(['Equity_US', 'Equity_UK', 'Equity_JP', 'Equity_HK', 'Equity_CA',
       'Equity_EU', 'Equity_CH', 'Equity_AU', 'Equity_NZ', 'Currency_UK',
       'Currency_JP', 'Currency_EU', 'Currency_HK', 'Currency_CH',
       'Currency_CA', 'Currency_AU', 'Currency_NZ', 'IR Future_SFRA Comdty',
       'IR Future_SFR1YZ2 Comdty', 'IR Future_SFR1YZ3 Comdty',
       'IR Future_SFR1YZ4 Comdty', 'IR Future_SFR1YZ5 Comdty',
       'IR Future_SFR1YZ6 Comdty', 'IR Future_SFR2YZ2 Comdty',
       'BondYield2Y_US', 'BondYield2Y_UK', 'BondYield2Y_JP', 'BondYield2Y_HK',
       'BondYield2Y_CA', 'BondYield2Y_CH', 'BondYield2Y_AU', 'BondYield2Y_NZ',
       'BondYield2Y_EU', 'BondYield10Y_US', 'BondYield10Y_UK',
       'BondYield10Y_JP', 'BondYield10Y_HK', 'BondYield10Y_CA',
       'BondYield10Y_CH', 'BondYield10Y_AU', 'BondYield10Y_NZ'],
      dtype='object')

In [68]:
def transform_currency_returns(ar_df):
    """
    Transform currency columns into asset returns in ar DataFrame.
    Handles JPY special case (already USD/JPY) and converts others to USD-based returns.
    
    Args:
        ar_df (pd.DataFrame): Asset returns DataFrame
        
    Returns:
        pd.DataFrame: Transformed DataFrame with currency returns
    """
    # Currency columns to process (excluding JPY)
    currency_cols = [col for col in ar_df.columns 
                   if col.startswith('Currency_') and not col.endswith('JP')]
    
    # JPY column (special handling)
    jpy_col = [col for col in ar_df.columns if col.endswith('JP')][0]
    
    # Transform non-JPY currencies (currently USD/FCY → need FCY/USD)
    for col in currency_cols:
        # Convert from USD/FCY to FCY/USD and calculate returns
        ar_df[col] = (1 / ar_df[col]).pct_change()
    
    # Transform JPY (currently USD/JPY → keep as is for returns)
    ar_df[jpy_col] = ar_df[jpy_col].pct_change()
    
    # Rename columns to reflect they're now returns
    ar_df.columns = [col.replace('Currency_', 'FXReturn_') for col in ar_df.columns]
    
    return ar_df


def create_us_centric_trade_factors(mf_df):
    """
    Create trade factors assuming each country primarily trades with the US.
    For US, creates an equally-weighted basket of all other currencies.
    
    Args:
        mf_df (pd.DataFrame): Macro factors DataFrame
        
    Returns:
        pd.DataFrame: Updated DataFrame with trade factors for all countries
    """
    # Get all currency columns (USD per FCY)
    currency_cols = [col for col in mf_df.columns if col.startswith('Currency_')]
    countries = [col.split('_')[1] for col in currency_cols]
    
    # Create trade factors for each country
    for country in countries + ['US']:  # Include US separately
        if country == 'US':
            # For US: equally-weighted basket of all other currencies
            changes = []
            for col in currency_cols:
                other_country = col.split('_')[1]
                
                if other_country == 'JP':
                    changes.append(1/(1+mf_df[col])-1)  # JPY is USD/JPY
                else:
                    changes.append(mf_df[col])  # Others are FCY/USD
            if changes:
                mf_df[f'TradeFactor_US'] = pd.DataFrame(changes).mean()
        else:
            # For non-US countries: use their currency vs USD
            col = f'Currency_{country}'
            if country == 'JP':
                mf_df[f'TradeFactor_{country}'] = mf_df[col]
            else:
                mf_df[f'TradeFactor_{country}'] = mf_df[col]
    
    # Apply 1-year smoothing
    # trade_cols = [col for col in mf_df.columns if col.startswith('TradeFactor_')]
    # mf_df[trade_cols] = mf_df[trade_cols].rolling(window=12).mean()
    
    return mf_df


# Apply transformation on ar
ar = transform_currency_returns(ar.copy())

# Apply transformation on mf
mf = create_us_centric_trade_factors(mf.copy())

# Verify
print("Trade factor columns created:")
print([col for col in mf.columns if col.startswith('TradeFactor_')])
print("\nSample trade factors:")
print(mf.filter(like='TradeFactor_'))

Trade factor columns created:
['TradeFactor_UK', 'TradeFactor_JP', 'TradeFactor_EU', 'TradeFactor_HK', 'TradeFactor_CH', 'TradeFactor_CA', 'TradeFactor_AU', 'TradeFactor_NZ', 'TradeFactor_US']

Sample trade factors:
            TradeFactor_UK  TradeFactor_JP  TradeFactor_EU  TradeFactor_HK  \
Date                                                                         
1980-12-31        0.076854       -0.154806       -0.119753        0.037298   
1981-12-31       -0.198409        0.082226       -0.167910        0.105928   
1982-12-31       -0.155091        0.067789       -0.116784        0.138840   
1983-12-31       -0.102905       -0.012782       -0.156827        0.201080   
1984-12-31       -0.202205        0.085887       -0.135181        0.004754   
1985-12-31        0.247841       -0.204094        0.251335       -0.001279   
1986-12-31        0.025952       -0.209488        0.233543       -0.002305   
1987-12-31        0.272175       -0.234049        0.206539       -0.003593   
1988

In [69]:
print(mf.filter(like='Trade'))

            TradeFactor_UK  TradeFactor_JP  TradeFactor_EU  TradeFactor_HK  \
Date                                                                         
1980-12-31        0.076854       -0.154806       -0.119753        0.037298   
1981-12-31       -0.198409        0.082226       -0.167910        0.105928   
1982-12-31       -0.155091        0.067789       -0.116784        0.138840   
1983-12-31       -0.102905       -0.012782       -0.156827        0.201080   
1984-12-31       -0.202205        0.085887       -0.135181        0.004754   
1985-12-31        0.247841       -0.204094        0.251335       -0.001279   
1986-12-31        0.025952       -0.209488        0.233543       -0.002305   
1987-12-31        0.272175       -0.234049        0.206539       -0.003593   
1988-12-31       -0.039767        0.031340       -0.111178        0.005667   
1989-12-31       -0.109608        0.149940        0.042204        0.000192   
1990-12-31        0.196899       -0.055981        0.133219      

# 2. Contruct portfolio

## 2.1 Business Cycle theme portfolio

In [38]:
def standardize_weights(raw_weights):
    """
    More robust weight standardization with debugging
    """
    standardized = pd.DataFrame(index=raw_weights.index, columns=raw_weights.columns)
    
    for date, row in raw_weights.iterrows():
        if row.isna().all():
            continue
            
        # Calculate z-scores with minimum divisor
        row_std = row.std()
        divisor = row_std if row_std > 1e-8 else 1.0  # Prevent divide-by-zero
        z_scores = (row - row.mean()) / divisor
        
        # Initialize weights
        weights = pd.Series(0, index=z_scores.index)
        
        # Long positions (positive z-scores)
        long_mask = z_scores > 0
        if long_mask.any():
            long_weights = z_scores[long_mask]
            weights[long_mask] = long_weights / long_weights.sum()
        
        # Short positions (negative z-scores)
        short_mask = z_scores < 0
        if short_mask.any():
            short_weights = z_scores[short_mask]
            weights[short_mask] = short_weights / (-short_weights.sum())
        
        standardized.loc[date] = weights.values
    
    return standardized


def calculate_bc_momentum(mf_df):
    """
    Calculate Business Cycle momentum scores for each country using:
    - 50% 1-year GDP growth change
    - 50% 1-year CPI inflation change
    """
    # Calculate 1-year changes for GDP and CPI
    gdp_changes = mf_df.filter(like='GDP_').diff()
    cpi_changes = mf_df.filter(like='CPI_').diff()
    
    # Combine 50/50 with proper sign conventions
    bc_scores = {}
    for country in [col.split('_')[1] for col in mf_df.columns if col.startswith('GDP_')]:
        gdp_col = f'GDP_{country}'
        cpi_col = f'CPI_{country}'
        
        # GDP: Higher growth = positive signal
        gdp_signal = gdp_changes[gdp_col]
        
        # CPI: Higher inflation = negative signal (except for currencies)
        cpi_signal = -cpi_changes[cpi_col]
        
        # Combine 50/50
        bc_scores[country] = 0.5*gdp_signal + 0.5*cpi_signal
    
    return pd.DataFrame(bc_scores)


def create_asset_class_portfolio(asset_returns, bc_scores, asset_class, target_vol=0.01):
    """
    Modified version with correct weight standardization
    """
    # Get relevant assets for this class
    if asset_class == 'Equity':
        assets = [col for col in asset_returns.columns if col.startswith('Equity_')]
        countries = [col.split('_')[1] for col in assets]
        # Equities: Long growth/inflation decline
        raw_scores = bc_scores[countries]  # Get scores for relevant countries
        
    elif asset_class == 'FX':
        assets = [col for col in asset_returns.columns if col.startswith('FXReturn_')]
        countries = [col.split('_')[1] for col in assets]
        # FX: Long growth/inflation increase (Balassa-Samuelson)
        raw_scores = bc_scores[countries]
        
    elif asset_class in ['Bond2Y', 'Bond10Y', 'IRFutures']:
        prefix = {
            'Bond2Y': 'BondYield2Y_',
            'Bond10Y': 'BondYield10Y_',
            'IRFutures': 'IRFutures_'
        }[asset_class]
        assets = [col for col in asset_returns.columns if col.startswith(prefix)]
        countries = [col.split('_')[-1] for col in assets]
        # Fixed Income: Short growth/inflation increase
        raw_scores = -bc_scores[countries]
        
    # Create DataFrame of raw scores aligned with asset returns index
    raw_weights = pd.DataFrame(index=asset_returns.index, columns=assets)
    for asset, country in zip(assets, countries):
        raw_weights[asset] = raw_scores[country]
    
    # Standardize weights using row-wise z-scoring
    weights = standardize_weights(raw_weights)
    
    if len(assets) > 1:
        returns = asset_returns[assets].dropna()
        weights = weights.loc[returns.index]
        min_years = 5
        portfolio_vol = pd.Series(index=weights.index, dtype=float)
        
        for i in range(min_years, len(weights)):
            current_date = weights.index[i]
            lookback_dates = weights.index[i-min_years:i]
            w_current = weights.loc[current_date].values.reshape(-1, 1)
            hist_returns = returns.loc[lookback_dates]
            
            if len(hist_returns.dropna()) >= min_years:
                C = hist_returns.cov()
                
                # Convert covariance to numpy array if needed
                C_matrix = C.values if hasattr(C, 'values') else C
                
                try:
                    # Proper matrix multiplication and scalar conversion
                    var = float(w_current.T @ C_matrix @ w_current)
                    portfolio_vol.loc[current_date] = np.sqrt(var)
                except Exception as e:
                    print(f"Vol calc error at {current_date}: {str(e)}")
                    portfolio_vol.loc[current_date] = np.nan
        
        # Forward fill and apply scaling
        portfolio_vol = portfolio_vol.ffill()
        scaling_factors = target_vol / np.sqrt(portfolio_vol.replace(0, np.nan))
        weights = weights.mul(scaling_factors, axis=0)
    
    return weights


def construct_bc_portfolio(ar_df, mf_df):
    """
    Construct complete Business Cycle long-short portfolio across all asset classes
    """
    # Calculate BC momentum scores
    bc_scores = calculate_bc_momentum(mf_df)
    
    # Create portfolios for each asset class
    equity_weights = create_asset_class_portfolio(ar_df, bc_scores, 'Equity')
    fx_weights = create_asset_class_portfolio(ar_df, bc_scores, 'FX')
    bond2y_weights = create_asset_class_portfolio(ar_df, bc_scores, 'Bond2Y')
    bond10y_weights = create_asset_class_portfolio(ar_df, bc_scores, 'Bond10Y')
    ir_weights = create_asset_class_portfolio(ar_df, bc_scores, 'IRFutures')
    
    # Combine all weights
    all_weights = pd.concat([
        equity_weights,
        fx_weights,
        bond2y_weights/2,
        bond10y_weights/2,
        ir_weights
    ], axis=1).fillna(0)
    
    # Calculate portfolio returns
    portfolio_returns = (all_weights.shift(1) * ar_df[all_weights.columns]).sum(axis=1)
    
    return {
        'weights': all_weights,
        'returns': portfolio_returns,
        'components': {
            'Equity': equity_weights,
            'FX': fx_weights,
            'Bond2Y': bond2y_weights,
            'Bond10Y': bond10y_weights,
            'IRFutures': ir_weights
        }
    }

# Manage index
ar = ar[(ar.index.year >= 1980) & (ar.index.year <= 2025)]

# Construct the portfolio
bc_portfolio = construct_bc_portfolio(ar, mf)

# Example analysis
print("Business Cycle Portfolio Summary:")
print(f"Annualized Return: {bc_portfolio['returns'].mean():.2%}")
print(f"Annualized Volatility: {bc_portfolio['returns'].std():.2%}")
print("\nRecent Weights:")
print(bc_portfolio['weights'].iloc[-4:].T)

Business Cycle Portfolio Summary:
Annualized Return: 2.49%
Annualized Volatility: 11.00%

Recent Weights:
Date             2022-12-31  2023-12-31  2024-12-31  2025-12-31
Equity_US          0.000016    0.011411   -0.001166   -0.007694
Equity_UK         -0.001600   -0.010259    0.007421   -0.005423
Equity_JP          0.001156   -0.002722   -0.004601    0.002523
Equity_HK         -0.001695    0.018949   -0.003647   -0.008829
Equity_CA          0.000824   -0.001466   -0.000212   -0.000315
Equity_EU         -0.001030   -0.002408    0.004941   -0.000315
Equity_CH          0.001061   -0.007433   -0.000021    0.000820
Equity_AU          0.000871   -0.005234   -0.000594    0.005360
Equity_NZ          0.000396   -0.000838   -0.002120    0.013874
FXReturn_UK       -0.002774   -0.001922    0.001638   -0.000483
FXReturn_JP        0.002011   -0.000282   -0.001069    0.000118
FXReturn_EU       -0.001784   -0.000214    0.001079   -0.000097
FXReturn_HK       -0.002939    0.004433   -0.000854   -0.00074

## 2.2 International Trade theme portfolio

In [70]:
def calculate_it_momentum(mf_df):
    # It's no longer lagged like GDP
    it_factors = mf_df.filter(like='TradeFactor_')
    
    it_scores = {}
    for country in [col.split('_')[1] for col in mf_df.columns if col.startswith('TradeFactor_')]:
        it_col = f'TradeFactor_{country}'
        
        # Here sign is positive, control later
        it_signal = it_factors[it_col]
        it_scores[country] = -it_signal # Turn appreciation into deprecition
        
    return pd.DataFrame(it_scores)


def create_asset_class_portfolio(asset_returns, it_scores, asset_class, target_vol=0.01):
    """
    Modified version with correct weight standardization
    """
    # Get relevant assets for this class
    if asset_class == 'Equity':
        assets = [col for col in asset_returns.columns if col.startswith('Equity_')]
        countries = [col.split('_')[1] for col in assets]
        # Equities: Depre -> High trade -> good for equity
        raw_scores = it_scores[countries]  # Get scores for relevant countries
        
    elif asset_class == 'FX':
        assets = [col for col in asset_returns.columns if col.startswith('FXReturn_')]
        countries = [col.split('_')[1] for col in assets]
        # FX: Depre -> bearish for fx
        raw_scores = -it_scores[countries]
        
    elif asset_class in ['Bond2Y', 'Bond10Y', 'IRFutures']:
        prefix = {
            'Bond2Y': 'BondYield2Y_',
            'Bond10Y': 'BondYield10Y_',
            'IRFutures': 'IRFutures_'
        }[asset_class]
        assets = [col for col in asset_returns.columns if col.startswith(prefix)]
        countries = [col.split('_')[-1] for col in assets]
        # Fixed Income: Depre -> High trade -> Reduce inflation&economy -> possible rate decrease -> bullish for Fixed Income
        raw_scores = -it_scores[countries]
        
    # Create DataFrame of raw scores aligned with asset returns index
    raw_weights = pd.DataFrame(index=asset_returns.index, columns=assets)
    for asset, country in zip(assets, countries):
        raw_weights[asset] = raw_scores[country]
    
    # Standardize weights using row-wise z-scoring
    weights = standardize_weights(raw_weights)
    
    if len(assets) > 1:
        returns = asset_returns[assets].dropna()
        weights = weights.loc[returns.index]
        min_years = 5
        portfolio_vol = pd.Series(index=weights.index, dtype=float)
        
        for i in range(min_years, len(weights)):
            current_date = weights.index[i]
            lookback_dates = weights.index[i-min_years:i]
            w_current = weights.loc[current_date].values.reshape(-1, 1)
            hist_returns = returns.loc[lookback_dates]
            
            if len(hist_returns.dropna()) >= min_years:
                C = hist_returns.cov()
                
                # Convert covariance to numpy array if needed
                C_matrix = C.values if hasattr(C, 'values') else C
                
                try:
                    # Proper matrix multiplication and scalar conversion
                    var = float(w_current.T @ C_matrix @ w_current)
                    portfolio_vol.loc[current_date] = np.sqrt(var)
                except Exception as e:
                    print(f"Vol calc error at {current_date}: {str(e)}")
                    portfolio_vol.loc[current_date] = np.nan
        
        # Forward fill and apply scaling
        portfolio_vol = portfolio_vol.ffill()
        scaling_factors = target_vol / np.sqrt(portfolio_vol.replace(0, np.nan))
        weights = weights.mul(scaling_factors, axis=0)
    
    return weights


def construct_it_portfolio(ar_df, mf_df):
    """
    Construct complete International Trade long-short portfolio across all asset classes
    """
    # Calculate BC momentum scores
    it_scores = calculate_it_momentum(mf_df)
    
    # Create portfolios for each asset class
    equity_weights = create_asset_class_portfolio(ar_df, it_scores, 'Equity')
    fx_weights = create_asset_class_portfolio(ar_df, it_scores, 'FX')
    bond2y_weights = create_asset_class_portfolio(ar_df, it_scores, 'Bond2Y')
    bond10y_weights = create_asset_class_portfolio(ar_df, it_scores, 'Bond10Y')
    ir_weights = create_asset_class_portfolio(ar_df, it_scores, 'IRFutures')
    
    # Combine all weights
    all_weights = pd.concat([
        equity_weights,
        fx_weights,
        bond2y_weights/2,
        bond10y_weights/2,
        ir_weights
    ], axis=1).fillna(0)
    
    # Calculate portfolio returns
    portfolio_returns = (all_weights.shift(1) * ar_df[all_weights.columns]).sum(axis=1)
    
    return {
        'weights': all_weights,
        'returns': portfolio_returns,
        'components': {
            'Equity': equity_weights,
            'FX': fx_weights,
            'Bond2Y': bond2y_weights,
            'Bond10Y': bond10y_weights,
            'IRFutures': ir_weights
        }
    }

# Manage index
ar = ar[(ar.index.year >= 1980) & (ar.index.year <= 2025)]

# Construct the portfolio
it_portfolio = construct_it_portfolio(ar, mf)

# Example analysis
print("International Trade Portfolio Summary:")
print(f"Annualized Return: {it_portfolio['returns'].mean():.2%}")
print(f"Annualized Volatility: {it_portfolio['returns'].std():.2%}")
print("\nRecent Weights:")
print(it_portfolio['weights'].iloc[-4:].T)

International Trade Portfolio Summary:
Annualized Return: -2.23%
Annualized Volatility: 11.60%

Recent Weights:
Date             2022-12-31  2023-12-31  2024-12-31  2025-12-31
Equity_US          0.000316    0.001494    0.001007   -0.000830
Equity_UK          0.001025   -0.004620    0.000518   -0.002945
Equity_JP         -0.001649   -0.006649   -0.004963    0.004382
Equity_HK         -0.000144    0.000202    0.000043    0.000549
Equity_CA         -0.000924    0.002432   -0.003778    0.002532
Equity_EU          0.000498   -0.002555    0.002400   -0.003961
Equity_CH         -0.000275    0.008596   -0.003456    0.005108
Equity_AU          0.000535    0.000332    0.003632   -0.001966
Equity_NZ          0.000620    0.000768    0.004597   -0.002871
FXReturn_UK       -0.003046    0.002677   -0.000069    0.000431
FXReturn_JP        0.004608    0.003902    0.000516   -0.000605
FXReturn_EU       -0.001537    0.001430   -0.000270    0.000575
FXReturn_HK        0.000299   -0.000235   -0.000018   -0

## 2.3 Monetary Policy theme portfolio

In [ ]:
def calculate_mp_momentum(mf_df):
    # It's no longer lagged like GDP
    mp_factors = mf_df.filter(like='BondYield2Y_')
    
    mp_scores = {}
    for country in [col.split('_')[1] for col in mf_df.columns if col.startswith('BondYield2Y_')]:
        mp_col = f'BondYield2Y_{country}'
        
        # Here sign is positive, control later
        mp_signal = mp_factors[mp_col].diff() # Here signal is the change in yield so diffed
        mp_scores[country] = mp_signal
    
    return pd.DataFrame(mp_scores)


def create_asset_class_portfolio(asset_returns, mp_scores, asset_class, target_vol=0.01):
    """
    Modified version with correct weight standardization
    """
    # Get relevant assets for this class
    if asset_class == 'Equity':
        assets = [col for col in asset_returns.columns if col.startswith('Equity_')]
        countries = [col.split('_')[1] for col in assets]
        # Equities: Yield up -> stock val down -> bearish to equity
        raw_scores = -mp_scores[countries]  # Get scores for relevant countries
        
    elif asset_class == 'FX':
        assets = [col for col in asset_returns.columns if col.startswith('FXReturn_')]
        countries = [col.split('_')[1] for col in assets]
        # FX: Yield up -> bullish for currency
        raw_scores = mp_scores[countries]
        
    elif asset_class in ['Bond2Y', 'Bond10Y', 'IRFutures']:
        prefix = {
            'Bond2Y': 'BondYield2Y_',
            'Bond10Y': 'BondYield10Y_',
            'IRFutures': 'IRFutures_'
        }[asset_class]
        assets = [col for col in asset_returns.columns if col.startswith(prefix)]
        countries = [col.split('_')[-1] for col in assets]
        # Fixed Income: Yield up -> price for fi assets down -> bullish for fi
        raw_scores = -mp_scores[countries]
        
    # Create DataFrame of raw scores aligned wmph asset returns index
    raw_weights = pd.DataFrame(index=asset_returns.index, columns=assets)
    for asset, country in zip(assets, countries):
        raw_weights[asset] = raw_scores[country]
    
    # Standardize weights using row-wise z-scoring
    weights = standardize_weights(raw_weights)
    
    if len(assets) > 1:
        returns = asset_returns[assets].dropna()
        weights = weights.loc[returns.index]
        min_years = 5
        portfolio_vol = pd.Series(index=weights.index, dtype=float)
        
        for i in range(min_years, len(weights)):
            current_date = weights.index[i]
            lookback_dates = weights.index[i-min_years:i]
            w_current = weights.loc[current_date].values.reshape(-1, 1)
            hist_returns = returns.loc[lookback_dates]
            
            if len(hist_returns.dropna()) >= min_years:
                C = hist_returns.cov()
                
                # Convert covariance to numpy array if needed
                C_matrix = C.values if hasattr(C, 'values') else C
                
                try:
                    # Proper matrix multiplication and scalar conversion
                    var = float(w_current.T @ C_matrix @ w_current)
                    portfolio_vol.loc[current_date] = np.sqrt(var)
                except Exception as e:
                    print(f"Vol calc error at {current_date}: {str(e)}")
                    portfolio_vol.loc[current_date] = np.nan
        
        # Forward fill and apply scaling
        portfolio_vol = portfolio_vol.ffill()
        scaling_factors = target_vol / np.sqrt(portfolio_vol.replace(0, np.nan))
        weights = weights.mul(scaling_factors, axis=0)
    
    return weights


def construct_mp_portfolio(ar_df, mf_df):
    """
    Construct complete Business Cycle long-short portfolio across all asset classes
    """
    # Calculate BC momentum scores
    mp_scores = calculate_mp_momentum(mf_df)
    
    # Create portfolios for each asset class
    equity_weights = create_asset_class_portfolio(ar_df, mp_scores, 'Equity')
    fx_weights = create_asset_class_portfolio(ar_df, mp_scores, 'FX')
    bond2y_weights = create_asset_class_portfolio(ar_df, mp_scores, 'Bond2Y')
    bond10y_weights = create_asset_class_portfolio(ar_df, mp_scores, 'Bond10Y')
    ir_weights = create_asset_class_portfolio(ar_df, mp_scores, 'IRFutures')
    
    # Combine all weights
    all_weights = pd.concat([
        equity_weights,
        fx_weights,
        bond2y_weights,
        bond10y_weights,
        ir_weights
    ], axis=1).fillna(0)
    
    # Calculate portfolio returns
    portfolio_returns = (all_weights.shift(1) * ar_df[all_weights.columns]).sum(axis=1)
    
    return {
        'weights': all_weights,
        'returns': portfolio_returns,
        'components': {
            'Equity': equity_weights,
            'FX': fx_weights,
            'Bond2Y': bond2y_weights,
            'Bond10Y': bond10y_weights,
            'IRFutures': ir_weights
        }
    }

# Manage index
ar = ar[(ar.index.year >= 1980) & (ar.index.year <= 2025)]

# Construct the portfolio
mp_portfolio = construct_mp_portfolio(ar, mf)

# Example analysis
print("Monetary Policy Portfolio Summary:")
print(f"Annualized Return: {mp_portfolio['returns'].mean():.2%}")
print(f"Annualized Volatility: {mp_portfolio['returns'].std():.2%}")
print("\nRecent Weights:")
print(mp_portfolio['weights'].iloc[-4:].T)

Monetary Policy Portfolio Summary:
Annualized Return: 1.43%
Annualized Volatility: 9.81%

Recent Weights:
Date             2022-12-31  2023-12-31  2024-12-31  2025-12-31
Equity_US         -0.000815   -0.002289   -0.001549   -0.000645
Equity_UK         -0.000188   -0.005665   -0.002035   -0.002549
Equity_JP          0.001527    0.007689   -0.004747   -0.005628
Equity_HK         -0.000599    0.000340    0.000037   -0.000367
Equity_CA         -0.000549   -0.000733    0.000696    0.003813
Equity_EU          0.000971    0.001010    0.010653    0.002814
Equity_CH          0.000745    0.001905   -0.000178    0.001011
Equity_AU         -0.000408    0.000023   -0.004034   -0.001352
Equity_NZ         -0.000683   -0.002279    0.001157    0.002905
FXReturn_UK        0.000849    0.004022    0.000230    0.000510
FXReturn_JP       -0.004167   -0.005003    0.000509    0.001106
FXReturn_EU       -0.002541   -0.000489   -0.001077   -0.000530
FXReturn_HK        0.002050   -0.000036    0.000016    0.00008

In [73]:
mf

,ExcessReturn_US,ExcessReturn_UK,ExcessReturn_JP,ExcessReturn_HK,ExcessReturn_CA,ExcessReturn_EU,ExcessReturn_AU,ExcessReturn_NZ,ExcessReturn_CH,GDP_AU,...,BondYield2Y_EU,TradeFactor_UK,TradeFactor_JP,TradeFactor_EU,TradeFactor_HK,TradeFactor_CH,TradeFactor_CA,TradeFactor_AU,TradeFactor_NZ,TradeFactor_US
Date,,,,,,,,,,,,,,,,,,,,,
1980-12-31,0.125026,-0.162977,-0.005504,0.545715,0.181234,-0.030000,-0.175000,-0.045000,0.007500,0.029,...,0.020420,0.076854,-0.154806,-0.119753,0.037298,0.120690,0.022338,0.068463,-0.021567,0.045936
1981-12-31,-0.253894,-0.131663,0.023459,-0.175990,-0.208626,-0.030000,-0.175000,-0.045000,0.007500,0.041,...,0.020420,-0.198409,0.082226,-0.167910,0.105928,0.000280,-0.006865,-0.045201,-0.144313,-0.066558
1982-12-31,0.027652,-0.119420,-0.011413,-0.572446,-0.068015,-0.030000,-0.175000,-0.045000,0.007500,0.001,...,0.020420,-0.155091,0.067789,-0.116784,0.138840,0.122763,0.036584,-0.131206,-0.109964,-0.034793
1983-12-31,0.080540,-0.098555,0.180040,-0.014883,0.233496,-0.030000,-0.175000,-0.045000,0.007500,-0.005,...,0.020420,-0.102905,-0.012782,-0.156827,0.201080,0.085430,0.012035,-0.079592,-0.106485,-0.016789
1984-12-31,-0.088044,0.135566,0.116647,0.243930,-0.129561,-0.030000,-0.175000,-0.045000,0.007500,0.063,...,0.020420,-0.202205,0.085887,-0.135181,0.004754,0.193667,0.062033,-0.082262,-0.273491,-0.063972
1985-12-31,0.182070,0.025667,0.083469,0.377690,0.138417,-0.030000,-0.175000,-0.045000,0.007500,0.055,...,0.020420,0.247841,-0.204094,0.251335,-0.001279,-0.207997,0.058107,-0.177458,0.050473,0.059682
1986-12-31,0.079681,0.081110,0.401469,0.394859,-0.012915,-0.030000,-0.175000,-0.045000,0.007500,0.024,...,0.020420,0.025952,-0.209488,0.233543,-0.002305,-0.216505,-0.012513,-0.023645,0.050050,0.039947
1987-12-31,-0.045521,-0.075893,0.120054,-0.171020,-0.039385,-0.310511,-0.175000,-0.045000,0.007500,0.049,...,0.020420,0.272175,-0.234049,0.206539,-0.003593,-0.212206,-0.058508,0.086793,0.257007,0.106722
1988-12-31,0.049077,-0.052623,0.373581,0.088177,0.002765,0.298993,-0.175000,-0.045000,-0.040328,0.043,...,0.020420,-0.039767,0.031340,-0.111178,0.005667,0.181282,-0.082603,0.184083,-0.041408,0.008211


## 2.4 Risk Sentiment theme portfolio

In [41]:
def calculate_rs_momentum(mf_df):
    # It's no longer lagged like GDP
    rs_factors = mf_df.filter(like='ExcessReturn_')
    
    rs_scores = {}
    for country in [col.split('_')[1] for col in mf_df.columns if col.startswith('ExcessReturn_')]:
        rs_col = f'ExcessReturn_{country}'
        
        # Here sign is positive, control later
        rs_signal = rs_factors[rs_col].diff() # Here signal is the change in yield so diffed
        rs_scores[country] = rs_signal
    
    return pd.DataFrame(rs_scores)


def create_asset_class_portfolio(asset_returns, rs_scores, asset_class, target_vol=0.01):
    """
    Modified version with correct weight standardization
    """
    # Get relevant assets for this class
    if asset_class == 'Equity':
        assets = [col for col in asset_returns.columns if col.startswith('Equity_')]
        countries = [col.split('_')[1] for col in assets]
        # Equities: Yield up -> stock val down -> bearish to equity
        raw_scores = -rs_scores[countries]  # Get scores for relevant countries
        
    elif asset_class == 'FX':
        assets = [col for col in asset_returns.columns if col.startswith('FXReturn_')]
        countries = [col.split('_')[1] for col in assets]
        # FX: Yield up -> bullish for currency
        raw_scores = rs_scores[countries]
        
    elif asset_class in ['Bond2Y', 'Bond10Y', 'IRFutures']:
        prefix = {
            'Bond2Y': 'BondYield2Y_',
            'Bond10Y': 'BondYield10Y_',
            'IRFutures': 'IRFutures_'
        }[asset_class]
        assets = [col for col in asset_returns.columns if col.startswith(prefix)]
        countries = [col.split('_')[-1] for col in assets]
        # Fixed Income: Yield up -> price for fi assets down -> bullish for fi
        raw_scores = -rs_scores[countries]
        
    # Create DataFrame of raw scores aligned with asset returns index
    raw_weights = pd.DataFrame(index=asset_returns.index, columns=assets)
    for asset, country in zip(assets, countries):
        raw_weights[asset] = raw_scores[country]
    
    # Standardize weights using row-wise z-scoring
    weights = standardize_weights(raw_weights)
    
    if len(assets) > 1:
        returns = asset_returns[assets].dropna()
        weights = weights.loc[returns.index]
        min_years = 5
        portfolio_vol = pd.Series(index=weights.index, dtype=float)
        
        for i in range(min_years, len(weights)):
            current_date = weights.index[i]
            lookback_dates = weights.index[i-min_years:i]
            w_current = weights.loc[current_date].values.reshape(-1, 1)
            hist_returns = returns.loc[lookback_dates]
            
            if len(hist_returns.dropna()) >= min_years:
                C = hist_returns.cov()
                
                # Convert covariance to numpy array if needed
                C_matrix = C.values if hasattr(C, 'values') else C
                
                try:
                    # Proper matrix multiplication and scalar conversion
                    var = float(w_current.T @ C_matrix @ w_current)
                    portfolio_vol.loc[current_date] = np.sqrt(var)
                except Exception as e:
                    print(f"Vol calc error at {current_date}: {str(e)}")
                    portfolio_vol.loc[current_date] = np.nan
        
        # Forward fill and apply scaling
        portfolio_vol = portfolio_vol.ffill()
        scaling_factors = target_vol / np.sqrt(portfolio_vol.replace(0, np.nan))
        weights = weights.mul(scaling_factors, axis=0)
    
    return weights


def construct_rs_portfolio(ar_df, mf_df):
    """
    Construct complete Business Cycle long-short portfolio across all asset classes
    """
    # Calculate BC momentum scores
    rs_scores = calculate_rs_momentum(mf_df)
    
    # Create portfolios for each asset class
    equity_weights = create_asset_class_portfolio(ar_df, rs_scores, 'Equity')
    fx_weights = create_asset_class_portfolio(ar_df, rs_scores, 'FX')
    bond2y_weights = create_asset_class_portfolio(ar_df, rs_scores, 'Bond2Y')
    bond10y_weights = create_asset_class_portfolio(ar_df, rs_scores, 'Bond10Y')
    ir_weights = create_asset_class_portfolio(ar_df, rs_scores, 'IRFutures')
    
    # Combine all weights
    all_weights = pd.concat([
        equity_weights,
        fx_weights,
        bond2y_weights,
        bond10y_weights,
        ir_weights
    ], axis=1).fillna(0)
    
    # Calculate portfolio returns
    portfolio_returns = (all_weights.shift(1) * ar_df[all_weights.columns]).sum(axis=1)
    
    return {
        'weights': all_weights,
        'returns': portfolio_returns,
        'components': {
            'Equity': equity_weights,
            'FX': fx_weights,
            'Bond2Y': bond2y_weights,
            'Bond10Y': bond10y_weights,
            'IRFutures': ir_weights
        }
    }

# Manage index
ar = ar[(ar.index.year >= 1980) & (ar.index.year <= 2025)]

# Construct the portfolio
rs_portfolio = construct_rs_portfolio(ar, mf)

# Example analysis
print("Monetary Policy Portfolio Summary:")
print(f"Annualized Return: {rs_portfolio['returns'].mean():.2%}")
print(f"Annualized Volatility: {rs_portfolio['returns'].std():.2%}")
print("\nRecent Weights:")
print(rs_portfolio['weights'].iloc[-4:].T)

Monetary Policy Portfolio Summary:
Annualized Return: 1.17%
Annualized Volatility: 3.42%

Recent Weights:
Date             2022-12-31  2023-12-31  2024-12-31  2025-12-31
Equity_US          0.002301   -0.006043    0.001558    0.004257
Equity_UK         -0.000864    0.004958    0.000663   -0.002559
Equity_JP         -0.000910   -0.005301    0.004484    0.004777
Equity_HK         -0.002139    0.004424   -0.010032   -0.001760
Equity_CA          0.000789    0.001084   -0.002399    0.001342
Equity_EU          0.000909   -0.002625    0.005172   -0.003647
Equity_CH          0.001300   -0.000191    0.000999   -0.003021
Equity_AU         -0.000396    0.001969    0.001485   -0.000688
Equity_NZ         -0.000989    0.001725   -0.001930    0.001298
FXReturn_UK        0.000910   -0.003283   -0.000132    0.001399
FXReturn_JP        0.000982    0.004732   -0.000717   -0.003665
FXReturn_EU       -0.001888    0.002641   -0.000823    0.002150
FXReturn_HK        0.002922   -0.002866    0.001508    0.00084

In [45]:
rs_portfolio['weights']

,Equity_US,Equity_UK,Equity_JP,Equity_HK,Equity_CA,Equity_EU,Equity_CH,Equity_AU,Equity_NZ,FXReturn_UK,...,BondYield10Y_NZ,IRFutures_US,IRFutures_UK,IRFutures_JP,IRFutures_HK,IRFutures_CA,IRFutures_EU,IRFutures_CH,IRFutures_AU,IRFutures_NZ
Date,,,,,,,,,,,,,,,,,,,,,
1980-12-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1981-12-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1982-12-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1983-12-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1984-12-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1985-12-31,-0.007211,0.005754,0.003136,-0.002559,-0.007138,0.002004,0.002004,0.002004,0.002004,-0.000912,...,0.019827,-0.064992,0.051860,0.028268,-0.023065,-0.064336,0.018066,0.018066,0.018066,0.018066
1986-12-31,0.002587,-0.000885,-0.006662,-0.000043,0.003664,0.000335,0.000335,0.000335,0.000335,0.000197,...,0.007272,0.045305,-0.015500,-0.116650,-0.000754,0.064161,0.005860,0.005860,0.005860,0.005860
1987-12-31,-0.000766,-0.000058,0.002712,0.009045,-0.002964,0.002692,-0.003553,-0.003553,-0.003553,0.000031,...,-0.095449,-0.009137,-0.000692,0.032346,0.107886,-0.035355,0.032106,-0.042385,-0.042385,-0.042385
1988-12-31,0.000950,0.002542,-0.002598,-0.002724,0.002121,-0.010545,0.004130,0.003062,0.003062,-0.000887,...,0.070986,0.014950,0.040004,-0.040876,-0.042867,0.033372,-0.165915,0.064978,0.048178,0.048178


In [48]:
rs_portfolio['returns']

Date
1980-12-31    0.000000
1981-12-31    0.000000
1982-12-31    0.000000
1983-12-31    0.000000
1984-12-31    0.000000
1985-12-31    0.000000
1986-12-31   -0.017395
1987-12-31    0.015998
1988-12-31    0.006840
1989-12-31    0.016300
1990-12-31   -0.022295
1991-12-31   -0.011840
1992-12-31   -0.009485
1993-12-31    0.015361
1994-12-31   -0.006231
1995-12-31    0.022567
1996-12-31   -0.035615
1997-12-31    0.013764
1998-12-31    0.025655
1999-12-31    0.017656
2000-12-31   -0.028446
2001-12-31    0.064135
2002-12-31   -0.002467
2003-12-31    0.011272
2004-12-31   -0.017977
2005-12-31    0.003302
2006-12-31    0.019387
2007-12-31   -0.022581
2008-12-31    0.004124
2009-12-31    0.084456
2010-12-31    0.001001
2011-12-31    0.061543
2012-12-31   -0.005642
2013-12-31   -0.017440
2014-12-31    0.002831
2015-12-31   -0.000112
2016-12-31    0.041486
2017-12-31    0.146539
2018-12-31    0.022628
2019-12-31    0.028309
2020-12-31   -0.004768
2021-12-31    0.018847
2022-12-31   -0.016326
2023-1

# 3. Integration of portfolios

In [42]:
print(bc_portfolio['components'].keys())

dict_keys(['Equity', 'FX', 'Bond2Y', 'Bond10Y', 'IRFutures'])


In [49]:
# List of portfolios to combine
portfolios = [bc_portfolio, it_portfolio, mp_portfolio, rs_portfolio]


In [51]:
bc_portfolio.keys()

dict_keys(['weights', 'returns', 'components'])

In [57]:
bc_portfolio['returns'].to_csv('bc_portfolio_returns.csv')
it_portfolio['returns'].to_csv('it_portfolio_returns.csv')
mp_portfolio['returns'].to_csv('mp_portfolio_returns.csv')
rs_portfolio['returns'].to_csv('rs_portfolio_returns.csv')

In [55]:
import pickle

for p in portfolios:
    p['returns'].to_csv(f"{data_dir}portfolio_{p['returns'].name}.csv")

In [ ]:

# === Combine 'weights' and 'returns' ===
final_weights = portfolios[0]['weights'].copy()
final_returns = portfolios[0]['returns'].copy()

for p in portfolios[1:]:
    final_weights += p['weights']
    final_returns += p['returns']

# === Combine 'components' ===
component_keys = portfolios[0]['components'].keys()
final_components = {}

for key in component_keys:
    # Start with a copy of the first portfolio's component
    combined_df = portfolios[0]['components'][key].copy()
    # Add the others
    for p in portfolios[1:]:
        combined_df += p['components'][key]
    final_components[key] = combined_df

# === Construct the final portfolio ===
final_portfolio = {
    'weights': final_weights,
    'returns': final_returns,
    'components': final_components
}


# Example analysis
print("Final Portfolio Summary:")
print(f"Annualized Return: {final_portfolio['returns'].mean():.2%}")
print(f"Annualized Volatility: {final_portfolio['returns'].std():.2%}")
print("\nRecent Weights:")
print(final_portfolio['weights'].iloc[-4:].T)